In [1]:
import pandas as pd
import sqlite3


In [2]:
conn = sqlite3.connect('../unzippedData/im.db')

In [3]:
! ls ../unzippedData

bom.movie_gross.csv  rt.movie_info.tsv    tmdb.movies.csv
im.db                rt.reviews.tsv       tn.movie_budgets.csv


In [4]:
# Function for Cleaning Column headers

def clean_column_headers(dataframe):
    dataframe.columns = dataframe.columns.str.title()
    dataframe.columns = dataframe.columns.str.replace("_", " ")

In [5]:
# Function for cleaning comma seperated string values of a series into list of strings. Accepts one series.

def clean_comma_series(Series):
    Series = Series.str.replace("_", " ")
    Series = Series.str.title()
    Series = Series.str.split(",")
    return Series

In [6]:
cursor = conn.cursor()

![Image of ERD Diagram IMDB](https://raw.githubusercontent.com/learn-co-curriculum/dsc-phase-2-project-v3/main/movie_data_erd.jpeg)


In [7]:
# displaying table

imdb_query = """ 
    SELECT *
    FROM sqlite_master
"""
imdb_df = pd.read_sql(imdb_query, conn)
imdb_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [8]:
# Connecting to Persons Table

persons_query = """ 
    SELECT *
    FROM persons
"""

persons_df = pd.read_sql(persons_query, conn)

# Data Cleaning

clean_column_headers(persons_df)
persons_df['Primary Profession'] = clean_comma_series(persons_df['Primary Profession'])
persons_df.drop(persons_df[persons_df['Death Year'].isna()==False].index, inplace=True)
persons_df.drop(persons_df[persons_df['Birth Year']<1930].index, inplace=True)
persons_df

,Person Id,Primary Name,Birth Year,Death Year,Primary Profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"[Miscellaneous, Production Manager, Producer]"
1,nm0061865,Joseph Bauer,NaN,NaN,"[Composer, Music Department, Sound Department]"
2,nm0062070,Bruce Baum,NaN,NaN,"[Miscellaneous, Actor, Writer]"
3,nm0062195,Axel Baumann,NaN,NaN,"[Camera Department, Cinematographer, Art Depar..."
4,nm0062798,Pete Baxter,NaN,NaN,"[Production Designer, Art Department, Set Deco..."
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,[Actress]
606644,nm9990690,Joo Yeon So,NaN,NaN,[Actress]
606645,nm9991320,Madeline Smith,NaN,NaN,[Actress]
606646,nm9991786,Michelle Modigliani,NaN,NaN,[Producer]


In [9]:
# Connecting to Principals Table

principals_query = """ 
    SELECT *
    FROM principals
"""

principals_df = pd.read_sql(principals_query, conn)
clean_column_headers(principals_df)

principals_df.head()

,Movie Id,Ordering,Person Id,Category,Job,Characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


In [10]:
# Connecting to Known_For Table

known_for_query = """ 
    SELECT *
    FROM known_for
"""

known_for_df = pd.read_sql(known_for_query, conn)
clean_column_headers(known_for_df)

known_for_df.head()

,Person Id,Movie Id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534


In [11]:
# Connecting to Movie Basics Table

movie_basics_query = """ 
    SELECT *
    FROM movie_basics
"""

movie_basics_df = pd.read_sql(movie_basics_query, conn)
clean_column_headers(movie_basics_df)

# movie_basics_df[(movie_basics_df['Primary Title'].str.contains('Paranormal Activity'))]
movie_basics_df['Start Year'].value_counts()

2017    17504
2016    17272
2018    16849
2015    16243
2014    15589
2013    14709
2012    13787
2011    12900
2010    11849
2019     8379
2020      937
2021       83
2022       32
2023        5
2024        2
2026        1
2025        1
2115        1
2027        1
Name: Start Year, dtype: int64

In [12]:
# Connecting to Movie Ratings Table

movie_ratings_query = """ 
    SELECT *
    FROM movie_ratings
"""
movie_ratings_df = pd.read_sql(movie_ratings_query, conn)
clean_column_headers(movie_ratings_df)

movie_ratings_df

,Movie Id,Averagerating,Numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [13]:
persons_df['Death Year'].value_counts()

Series([], Name: Death Year, dtype: int64)

In [14]:
# dropping dead people. no pun intended
persons_df.drop(persons_df[persons_df['Death Year'].isna()==False].index, inplace=True)
persons_df

,Person Id,Primary Name,Birth Year,Death Year,Primary Profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"[Miscellaneous, Production Manager, Producer]"
1,nm0061865,Joseph Bauer,NaN,NaN,"[Composer, Music Department, Sound Department]"
2,nm0062070,Bruce Baum,NaN,NaN,"[Miscellaneous, Actor, Writer]"
3,nm0062195,Axel Baumann,NaN,NaN,"[Camera Department, Cinematographer, Art Depar..."
4,nm0062798,Pete Baxter,NaN,NaN,"[Production Designer, Art Department, Set Deco..."
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,[Actress]
606644,nm9990690,Joo Yeon So,NaN,NaN,[Actress]
606645,nm9991320,Madeline Smith,NaN,NaN,[Actress]
606646,nm9991786,Michelle Modigliani,NaN,NaN,[Producer]


In [17]:
# Connecting to Movie Akas Table

movie_akas_query = """ 
    SELECT *
    FROM movie_akas
"""
movie_akas_df = pd.read_sql(movie_akas_query, conn)
clean_column_headers(movie_akas_df)
movie_akas_df

,Movie Id,Ordering,Title,Region,Language,Types,Attributes,Is Original Title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [23]:
# Connecting to Movie Basics Table

movie_basics_query = """ 
    SELECT *
    FROM movie_basics
"""
movie_basics_df = pd.read_sql(movie_basics_query, conn)
clean_column_headers(movie_basics_df)
movie_basics_df['Genres'] = clean_comma_series(movie_basics_df['Genres'])
movie_basics_df


,Movie Id,Primary Title,Original Title,Start Year,Runtime Minutes,Genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"[Action, Crime, Drama]"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"[Biography, Drama]"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,[Drama]
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"[Comedy, Drama]"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"[Comedy, Drama, Fantasy]"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,[Drama]
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,[Documentary]
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,[Comedy]
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None
